In [1]:
import json

import nltk
import pandas as pd
import torch as tt

nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.tokenize import sent_tokenize
from tqdm import tqdm_notebook
from math import ceil

In [3]:
import evaluate

In [4]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedModel, PreTrainedTokenizer
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
from typing import Any, Dict

In [5]:
# models:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2").to(tt.device("cuda:0"))
tokenizer.pad_token = tokenizer.eos_token

# metrics:
bleu4 = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

/usr/lib/python3/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
muserc_train = pd.read_json("MuSeRC/train.jsonl", lines=True).to_dict(orient="records")
muserc_val = pd.read_json("MuSeRC/val.jsonl", lines=True).to_dict(orient="records")

In [7]:
def to_dg_format(dataset: list[dict[str, Any]]) -> list[dict[str, Any]]:
    dataset_processed = []
    item_id = 0

    for iidx, item in enumerate(dataset):
        for question in item["passage"]["questions"]:
            new_item = {
                "item_id": item_id,
                "passage_id": item["idx"],
                "passage": item["passage"]["text"],
                "question": question["question"],
                "distractors": [f'"{answer["text"]}"' for answer in question["answers"] if answer["label"] == 0],
                "right_answer": [answer["text"] for answer in question["answers"] if answer["label"] == 1][0]
            }
            dataset_processed.append(new_item)
            item_id += 1

    return dataset_processed

In [8]:
muserc_train_dg = to_dg_format(muserc_train)

In [9]:
muserc_train_dg[0]

{'item_id': 0,
 'passage_id': 0,
 'passage': '(1) Но люди не могут существовать без природы, поэтому в парке стояли железобетонные скамейки — деревянные моментально ломали. (2) В парке бегали ребятишки, водилась шпана, которая развлекалась игрой в карты, пьянкой, драками, «иногда насмерть». (3) «Имали они тут и девок...» (4) Верховодил шпаной Артемка-мыло, с вспененной белой головой. (5) Людочка сколько ни пыталась усмирить лохмотья на буйной голове Артемки, ничего у неё не получалось. (6) Его «кудри, издали напоминавшие мыльную пену, изблизя оказались что липкие рожки из вокзальной столовой — сварили их, бросили комком в пустую тарелку, так они, слипшиеся, неподъёмно и лежали. (7) Да и не ради причёски приходил парень к Людочке. (8) Как только её руки становились занятыми ножницами и расчёской, Артемка начинал хватать её за разные места. (9) Людочка сначала увёртывалась от хватких рук Артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на

In [10]:
muserc_val_dg = to_dg_format(muserc_val)

In [11]:
muserc_val_dg[0]

{'item_id': 0,
 'passage_id': 0,
 'passage': '(1) Самый первый «остров» Архипелага возник в 1923 году на месте Соловецкого монастыря. (2) Затем появились ТОНы — тюрьмы особого назначения и этапы. (3) Люди попадали на Архипелаг разными способами: в вагон-заках, на баржах, пароходах и пешими этапами. (4) В тюрьмы арестованных доставляли в «воронках» — фургончиках чёрного цвета. (5) Роль портов Архипелага играли пересылки, временные лагеря, состоящие из палаток, землянок, бараков или участков земли под открытым небом. (6) На всех пересылках держать «политических» в узде помогали специально отобранные урки, или «социально близкие». (7) Солженицын побывал на пересылке Красная Пресня в 1945 году. (8) Эмигранты, крестьяне и «малые народы» перевозили красными эшелонами. (9) Чаще всего такие эшелоны останав\xadливались на пустом месте, посреди степи или тайги, и осуждённые сами строили лагерь. (10) Особо важные заключённые, в основном учёные, перевозились спецконвоем. (11) Так перевозили и Солж

In [12]:
LB = "\n  "

def to_dg_format_final(dataset: list[dict[str, Any]]) -> list[dict[str, Any]]:
    new_dataset = []

    for item in dataset:
        new_item = {
            "item_id": item["item_id"],
            "passage_id": item["passage_id"],
            "inp": f'{item["passage"]} ВОПРОС: {item["question"]} ПРАВИЛЬНЫЙ ОТВЕТ: {item["right_answer"]} НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:',
            "outp_expected": f'{item["passage"]} ВОПРОС: {item["question"]} ПРАВИЛЬНЫЙ ОТВЕТ: {item["right_answer"]} НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:{LB + LB.join(item["distractors"])}',
            "right_answer": item["right_answer"],
            "distractors": LB + LB.join(item["distractors"])
        }
        new_dataset.append(new_item)

    return new_dataset

In [13]:
muserc_train_dg = Dataset.from_list(to_dg_format_final(muserc_train_dg))
muserc_val_dg = Dataset.from_list(to_dg_format_final(muserc_val_dg))

In [14]:
def cut_last_break(input_: list[str]) -> list[str]:
    output = [s[:s.rfind('\n')] for s in input_]
    return output

def parse_options(input_: list[str]) -> list[str]:
    output = [s.strip() for s in input_]
    output = [set(option.strip() for option in s.split('\n')) for s in output]
    output = [sorted(list(s))[:3] for s in output]
    output = ['\n'.join(s) for s in output]
    return output

def get_metric_inputs(
    input_batch: list[str], label_batch: list[str],
    model: PreTrainedModel, tokenizer: PreTrainedTokenizer
) -> list[str]:
    FACTOR = 1.1

    input_batch_ = tokenizer(input_batch, return_tensors="pt", padding=True)["input_ids"].to(tt.device("cuda:0"))
    label_batch_ = tokenizer(label_batch, return_tensors="pt", padding=True)["input_ids"]

    input_length = input_batch_.shape[-1]
    output_length = label_batch_.shape[-1]
    
    with tt.no_grad():
        output_batch = model.generate(input_batch_, max_length=input_length + ceil(output_length * FACTOR))
        output_batch = output_batch[:,input_length:]

    output = tokenizer.batch_decode(output_batch)
    del input_batch_
    del output_batch
    del label_batch_
    tt.cuda.empty_cache()

    output = cut_last_break(output)
    output = parse_options(output)

    return output

def compute_metrics(output: list[str], label_batch: list[str]) -> dict[str, Any]:
    metric_dict = {
        "bleu": bleu4.compute(predictions=output, references=[[label] for label in label_batch]),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

In [15]:
def compute_metrics_on_dataset(
    dataset: Dataset, model: PreTrainedModel=model,
    tokenizer: PreTrainedTokenizer=tokenizer
) -> pd.DataFrame:
    batch_size = 1

    n_steps = (len(dataset) // batch_size) + 1
    metrics = []

    for i in tqdm_notebook(range(n_steps), total=n_steps):
        slice = dataset[i*batch_size:(i+1)*batch_size]
        if slice["inp"]:
            distractors = slice["distractors"]
            
            output = get_metric_inputs(slice["inp"], distractors, model, tokenizer)

            distractors = parse_options(distractors)
    
            if len(distractors[0]) > 0:
                metric = compute_metrics(output, distractors)
                metrics.append({
                    "item_id": slice["item_id"][0],
                    "passage_id": slice["passage_id"][0],
                    "inp": slice["inp"][0],
                    "distractors": distractors[0],
                    
                    "output": output[0],
        
                    "bleu": metric["bleu"]["bleu"],
                    "sbleu": metric["sbleu"]["score"],
                    "rouge1": metric["rouge"]["rouge1"],
                    "rouge2": metric["rouge"]["rouge2"],
                    "rougeL": metric["rouge"]["rougeL"],
                    "rougeLsum": metric["rouge"]["rougeLsum"],
                    "meteor": metric["meteor"]["meteor"],
                })

    return pd.DataFrame(metrics)

In [16]:
METRIC_COLS = [
    "bleu", "sbleu", "rouge1", "rouge2",
    "rougeL", "rougeLsum", "meteor"
]

In [17]:
metrics_muserc_train = compute_metrics_on_dataset(muserc_train_dg)

/tmp/ipykernel_67073/2888913650.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n_steps), total=n_steps):


  0%|          | 0/2898 [00:00<?, ?it/s]

In [18]:
metrics_muserc_train[METRIC_COLS].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,2896.000000,2896.000000,2896.000000,2896.000000,2896.000000,2896.000000,2896.000000
mean,0.013021,4.514656,0.016419,0.004271,0.015882,0.016154,0.112488
std,0.059654,6.094096,0.098442,0.050239,0.095903,0.097252,0.104559
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.819687,0.000000,0.000000,0.000000,0.000000,0.052083
50%,0.000000,2.853184,0.000000,0.000000,0.000000,0.000000,0.084746
75%,0.000000,4.673290,0.000000,0.000000,0.000000,0.000000,0.128205
max,0.753520,75.352011,1.000000,1.000000,1.000000,1.000000,0.845734


In [19]:
metrics_muserc_val = compute_metrics_on_dataset(muserc_val_dg)

/tmp/ipykernel_67073/2888913650.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n_steps), total=n_steps):


  0%|          | 0/530 [00:00<?, ?it/s]

In [20]:
metrics_muserc_val[METRIC_COLS].describe()

,bleu,sbleu,rouge1,rouge2,rougeL,rougeLsum,meteor
count,528.000000,528.000000,528.000000,528.000000,528.000000,528.000000,528.000000
mean,0.015886,4.577642,0.014340,0.005303,0.014340,0.014340,0.112546
std,0.067472,6.491029,0.087562,0.055270,0.087562,0.087562,0.107736
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.888734,0.000000,0.000000,0.000000,0.000000,0.050463
50%,0.000000,2.860842,0.000000,0.000000,0.000000,0.000000,0.083160
75%,0.000000,4.799811,0.000000,0.000000,0.000000,0.000000,0.127715
max,0.673319,67.331875,1.000000,1.000000,1.000000,1.000000,0.823937


In [21]:
metrics_muserc_train.to_csv("metrics_muserc_train_gpt3_baseline.csv", sep=';')
metrics_muserc_val.to_csv("metrics_muserc_val_gpt3_baseline.csv", sep=';')